In [57]:
# test
x = 23
println("Hello World $x")

Hello World 23


In [58]:
using DataFrames
using CSV
using DataFrames
using Statistics
using MLJ
using MLJDecisionTreeInterface

In [59]:
file_path = "titanic_dataset.csv"

"titanic_dataset.csv"

In [60]:
df = CSV.read(file_path, DataFrame)

Row,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,Int64,Int64,Int64,String,String7,Float64?,Int64,Int64,String31,Float64,String15?,String1?
1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,missing,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,missing,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,missing,S
6,6,0,3,"Moran, Mr. James",male,missing,0,0,330877,8.4583,missing,Q
7,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,missing,S
9,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,missing,S


In [61]:
# For simplicity, let's drop columns with a lot of missing values or non-numeric data
# You can perform more sophisticated preprocessing as needed
drop_columns = [:Name, :Ticket, :Cabin, :Embarked]
df = select(df, Not(drop_columns))

Row,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
,Int64,Int64,Int64,String7,Float64?,Int64,Int64,Float64
1,1,0,3,male,22.0,1,0,7.25
2,2,1,1,female,38.0,1,0,71.2833
3,3,1,3,female,26.0,0,0,7.925
4,4,1,1,female,35.0,1,0,53.1
5,5,0,3,male,35.0,0,0,8.05
6,6,0,3,male,missing,0,0,8.4583
7,7,0,1,male,54.0,0,0,51.8625
8,8,0,3,male,2.0,3,1,21.075
9,9,1,3,female,27.0,0,2,11.1333


In [62]:
# Handle missing values, for example by removing rows with missing values
df = dropmissing(df)

Row,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
,Int64,Int64,Int64,String7,Float64,Int64,Int64,Float64
1,1,0,3,male,22.0,1,0,7.25
2,2,1,1,female,38.0,1,0,71.2833
3,3,1,3,female,26.0,0,0,7.925
4,4,1,1,female,35.0,1,0,53.1
5,5,0,3,male,35.0,0,0,8.05
6,7,0,1,male,54.0,0,0,51.8625
7,8,0,3,male,2.0,3,1,21.075
8,9,1,3,female,27.0,0,2,11.1333
9,10,1,2,female,14.0,1,0,30.0708


In [63]:
# Convert categorical columns to proper MLJ format
categorical_columns = [:Sex, :Pclass]
for col in categorical_columns
    df[!, col] = categorical(df[!, col])
end

In [64]:
# split the data
y, X = unpack(df, ==(:Survived));  # Survived is the target variable
train, test = partition(eachindex(y), 0.7, shuffle=true)  # 70% train, 30% test

([510, 333, 424, 80, 541, 678, 86, 281, 704, 247  …  413, 622, 564, 561, 198, 604, 10, 284, 345, 307], [676, 634, 193, 448, 477, 95, 417, 475, 364, 111  …  539, 712, 563, 347, 536, 278, 630, 137, 61, 443])

In [66]:
# check if DecisionTree is available
for model in models()
    if occursin("DecisionTree", model.package_name)
        println(model.name)
    end
end

AdaBoostStumpClassifier
DecisionTreeClassifier
DecisionTreeRegressor


RandomForestClassifier
RandomForestRegressor


In [69]:
# Load the DecisionTreeClassifier
tree_model = @load DecisionTreeClassifier pkg=BetaML


import BetaML ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/abhi/.julia/packages/MLJModels/L1gIK/src/loading.jl:159


BetaML.Trees.DecisionTreeClassifier

In [73]:
tree_model = BetaML.DecisionTreeClassifier()

DecisionTreeClassifier(
  max_depth = 0, 
  min_gain = 0.0, 
  min_records = 2, 
  max_features = 0, 
  splitting_criterion = BetaML.Utils.gini, 
  rng = Random._GLOBAL_RNG())

In [76]:
# Train the model
tree = machine(tree_model, X, y)
MLJ.fit!(tree, rows=train)

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc BetaML.DecisionTreeClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{3}}, AbstractVector{Multiclass{2}}}}, AbstractVector{Count}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Union{Missing, Known}}}, AbstractVector{<:Union{Missing, Finite}}

trained Machine; caches model-specific representations of data
  model: DecisionTreeClassifier(max_depth = 0, …)
  args: 
    1:	Source @217 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{3}}, AbstractVector{Multiclass{2}}}}
    2:	Source @035 ⏎ AbstractVector{Count}


In [84]:
# Evaluate the model
y = categorical(y)
y_pred = MLJ.predict(tree, rows=test)

MethodError: MethodError: no method matching classes(::Int64)

Closest candidates are:
  classes(!Matched::CategoricalArrays.CategoricalPool)
   @ CategoricalDistributions ~/.julia/packages/CategoricalDistributions/UmKVV/src/utilities.jl:58
  classes(!Matched::CategoricalArrays.CategoricalValue)
   @ CategoricalDistributions ~/.julia/packages/CategoricalDistributions/UmKVV/src/utilities.jl:59
  classes(!Matched::CategoricalArrays.CategoricalArray)
   @ CategoricalDistributions ~/.julia/packages/CategoricalDistributions/UmKVV/src/utilities.jl:60
  ...
